In [1]:
from __future__ import print_function

import glob
import nibabel
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import skimage.transform
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [2]:
path = Path('.')

In [3]:
patients = glob.glob(str(path / '*' / '*'))
print(f"{len(patients)} patients found.")

60 patients found.


In [ ]:
def show_image(patient):
    patient = patients[patient]
    print(patient)
    t1 = nibabel.load(path / patient / 'pre' / 'T1.nii.gz').get_fdata()
    flair = nibabel.load(path / patient / 'pre' / 'FLAIR.nii.gz').get_fdata()
    gt = nibabel.load(path / patient / 'wmh.nii.gz').get_fdata()
    assert t1.shape == flair.shape and t1.shape == gt.shape
    def f(s):
        plt.subplot(1, 3, 1)
        plt.title('T1')
        plt.imshow(t1[:, :, s])
        plt.subplot(1, 3, 2)
        plt.title('FLAIR')
        plt.imshow(flair[:, :, s])
        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.imshow(gt[:, :, s])
        plt.show()
    return interact(f, s=IntSlider(min=0, max=t1.shape[-1]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(patients)-1, value=0))

In [ ]:
images = np.load('images.npy')
gts = np.load('gts.npy')

In [ ]:
def show_image(patient):
    print(patients[patient])
    def f(s):
        plt.subplot(1, 3, 1)
        plt.title('T1')
        plt.imshow(images[patient, :, :, s, 0])
        plt.subplot(1, 3, 2)
        plt.title('FLAIR')
        plt.imshow(images[patient, :, :, s, 1])
        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.imshow(gts[patient, :, :, s])
        plt.show()
    return interact(f, s=IntSlider(min=0, max=images.shape[-2]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(patients)-1, value=0))

In [16]:
import skimage
import skimage.io
import skimage.morphology

In [22]:
expected_shape = (200, 200, 48)
disk = skimage.morphology.disk(2.5)

def generate_data(save_dir):
    for patient in patients:
        print(patient)
        t1 = nibabel.load(path / patient / 'pre' / 'T1.nii.gz').get_fdata()
        fl = nibabel.load(path / patient / 'pre' / 'FLAIR.nii.gz').get_fdata()
        gt = nibabel.load(path / patient / 'wmh.nii.gz').get_fdata() != 0

        t1 = skimage.transform.resize(t1, expected_shape)
        fl = skimage.transform.resize(fl, expected_shape)
        gt = skimage.transform.resize(gt, expected_shape)

        slices_morph = None
        for s in range(48):
            flair_slice = fl[:, :, s]
            
        
            morph = skimage.morphology.dilation(flair_slice, disk)
            top_hat = skimage.util.invert(morph - flair_slice)
            top_hat = top_hat.reshape((200, 200, 1))
            if slices_morph is None:
                slices_morph = top_hat
            else:
                slices_morph = np.concatenate([slices_morph, top_hat], axis=-1)

        slices_morph = np.array(slices_morph)
        
        slices_morph = slices_morph.reshape((200, 200, 48, 1))

        t1 = t1.reshape((200, 200, 48, 1))
        fl = fl.reshape((200, 200, 48, 1))
        gt = gt.reshape((200, 200, 48, 1))
        
        img = np.concatenate([t1, fl], axis=-1)
        pre_processed = np.concatenate([img, slices_morph], axis=-1)
        
        n0 = patient.split('/')[1]
        
        np.save(save_dir + '/data/' + str(n0) + '.npy', img)
        np.save(save_dir + '/data/' + str(n0) + '_preprocessed' + '.npy', pre_processed)
        np.save(save_dir + '/labels/' + str(n0) + '.npy', gt)

In [37]:
x_train = np.load('../models/2d/X_train.npy')
x_test = np.load('../models/2d/X_test.npy')
y_train = np.load('../models/2d/Y_train.npy')
y_test = np.load('../models/2d/Y_test.npy')

In [40]:
transformed_x_train = [path[0].split('/')[4] for path in x_train]
transformed_y_train = [path.split('/')[4] for path in y_train]

transformed_x_test = [path[0].split('/')[4] for path in x_test]
transformed_y_test = [path.split('/')[4] for path in y_test]

In [41]:
np.save('X_train.npy', transformed_x_train)
np.save('X_test.npy', transformed_x_test)
np.save('Y_train.npy', transformed_y_train)
np.save('Y_test.npy', transformed_y_test)